# Table of skills by year

Generates a LaTeX table of skills for each forecast method over the years 2011-2018.

In [ ]:
# Autoreload packages that are modified
%load_ext autoreload
%autoreload 2

# Load relevant packages
import numpy as np
import pandas as pd
from sklearn import *
import sys
import subprocess
from datetime import datetime, timedelta
import netCDF4
import time
from functools import partial
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Ensure correct working directory
if os.path.basename(os.getcwd()) == "visualize":
    # Navigate to base folder
    os.chdir(os.path.join("..",".."))

# Adds 'experiments' folder to path to load experiments_util
sys.path.insert(0, 'src/experiments')
# Load general utility functions
from experiments_util import *

In [ ]:
gt_id = "contest_tmp2m" # "contest_tmp2m" or "contest_precip"

target_horizon = "34w"
backward34_file="results/skills/backward_stepwise/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)
knn34_file="results/skills/knn_target_regression/skill-{}-{}-days60-early{}.h5".format(gt_id, target_horizon, 337 if target_horizon=="34w" else 323)
ensemble34_file="results/skills/ensemble_backward_stepwise_and_knn/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)
deb_cfs34_file="results/skills/debiased_cfsv2/skill-{}-{}.h5".format(gt_id, target_horizon)
ens_cfs34_file="results/skills/ensemble_backward_stepwise_knn_debiased_cfsv2/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)

target_horizon = "56w"
backward56_file="results/skills/backward_stepwise/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)
knn56_file="results/skills/knn_target_regression/skill-{}-{}-days60-early{}.h5".format(gt_id, target_horizon, 337 if target_horizon=="34w" else 323)
ensemble56_file="results/skills/ensemble_backward_stepwise_and_knn/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)
deb_cfs56_file="results/skills/debiased_cfsv2/skill-{}-{}.h5".format(gt_id, target_horizon)
ens_cfs56_file="results/skills/ensemble_backward_stepwise_knn_debiased_cfsv2/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)

del target_horizon

In [ ]:
backward34_skills = pd.read_hdf(backward34_file)
knn34_skills = pd.read_hdf(knn34_file)
ensemble34_skills = pd.read_hdf(ensemble34_file)
deb_cfs34_skills = pd.read_hdf(deb_cfs34_file)
ens_cfs34_skills = pd.read_hdf(ens_cfs34_file)

backward56_skills = pd.read_hdf(backward56_file)
knn56_skills = pd.read_hdf(knn56_file)
ensemble56_skills = pd.read_hdf(ensemble56_file)
deb_cfs56_skills = pd.read_hdf(deb_cfs56_file)
ens_cfs56_skills = pd.read_hdf(ens_cfs56_file)

In [ ]:
all34_skills = pd.concat(
    [backward34_skills.start_date, 
     backward34_skills.skill, knn34_skills.skill, ensemble34_skills.skill, 
     deb_cfs34_skills.skill, ens_cfs34_skills.skill],
    keys=['start_date','backward34','knn34','ensemble34','deb_cfs34','ens_cfs34'
         ], axis=1)
all56_skills = pd.concat(
    [backward56_skills.start_date, 
     backward56_skills.skill, knn56_skills.skill, ensemble56_skills.skill,
     deb_cfs56_skills.skill, ens_cfs56_skills.skill],
    keys=['start_date','backward56','knn56','ensemble56','deb_cfs56','ens_cfs56'
         ], axis=1)

In [ ]:
print '\\begin{table*}[ht]'
print '\\begin{tabular}{c|ccccc|ccccc}'
print '\\toprule'
gt_str = 'temperature' if gt_id.endswith("tmp2m") else 'precipitation'
print '& \\multicolumn{5}{c|}{'+gt_str+', weeks 3-4} & \multicolumn{5}{c}{'+gt_str+', weeks 5-6} \\\\'
print '\\midrule'
print 'year &  multillr & autoknn & ensemble & rec-deb-cfs & ens-cfs &  multillr & autoknn & ensemble & rec-deb-cfs & ens-cfs \\\\'
print '\\midrule'
# Add individual year skills
years = range(2011, 2018)
for i in range(7):
    # Find relevant dates for this year
    skills34_year = all34_skills[
        (all34_skills.start_date >= get_target_date(str(years[i])+"0418", "34w")) & 
        (all34_skills.start_date <= get_target_date(str(years[i]+1)+"0417", "34w"))]
    backward34 = round(skills34_year.backward34.mean(), 4)
    knn34 = round(skills34_year.knn34.mean(), 4)
    ensemble34 = round(skills34_year.ensemble34.mean(), 4)
    deb_cfs34 = round(skills34_year.deb_cfs34.mean(), 4)
    ens_cfs34 = round(skills34_year.ens_cfs34.mean(), 4)
    skills56_year = all56_skills[
        (all56_skills.start_date >= get_target_date(str(years[i])+"0418", "56w")) & 
        (all56_skills.start_date <= get_target_date(str(years[i]+1)+"0417", "56w"))]
    backward56 = round(skills56_year.backward56.mean(), 4)
    knn56 = round(skills56_year.knn56.mean(), 4)
    ensemble56 = round(skills56_year.ensemble56.mean(), 4)
    deb_cfs56 = round(skills56_year.deb_cfs56.mean(), 4)
    ens_cfs56 = round(skills56_year.ens_cfs56.mean(), 4)
    print '{} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} \\\\'.format(
        str(years[i]), 
        backward34, knn34, ensemble34, deb_cfs34, ens_cfs34,
        backward56, knn56, ensemble56, deb_cfs56, ens_cfs56)
    
# Add overall means
print '\\hline'
backward34 = round(all34_skills.backward34.mean(), 4)
knn34 = round(all34_skills.knn34.mean(), 4)
ensemble34 = round(all34_skills.ensemble34.mean(), 4)
deb_cfs34 = round(all34_skills.deb_cfs34.mean(), 4)
ens_cfs34 = round(all34_skills.ens_cfs34.mean(), 4)
backward56 = round(all56_skills.backward56.mean(), 4)
knn56 = round(all56_skills.knn56.mean(), 4)
ensemble56 = round(all56_skills.ensemble56.mean(), 4)
deb_cfs56 = round(all56_skills.deb_cfs56.mean(), 4)
ens_cfs56 = round(all56_skills.ens_cfs56.mean(), 4)
print 'all & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} & {:0.4f} \\\\'.format(
    backward34, knn34, ensemble34, deb_cfs34, ens_cfs34,
    backward56, knn56, ensemble56, deb_cfs56, ens_cfs56)
print '\\bottomrule\n\\end{tabular}'
print '\\end{table*}'